In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras

In [41]:
# Read the 'fatiguelevel_updated.csv' file
data_df = pd.read_csv('fatiguelevel_updated.csv')

In [42]:
#Extract the file paths and labels

file_paths = data_df['file_loc'].values
labels = data_df['Stressed'].values


In [43]:
# Prepare the data
data = []
for file_path in file_paths:
    df = pd.read_csv(file_path)
    data.append(df)

In [44]:
# Convert the data to numpy array
data = np.array(data)

In [45]:

# Print the shapes of the input features and labels
print("Input features shape:", data.shape)
# print("Labels shape:", np.array(labels).shape)

Input features shape: (459, 720, 8)


In [52]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Split the training data into training and validation sets
# The validation size can be set to a specific percentage of the training data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [53]:
# Determine the shape of the input data
num_samples = data.shape[0]
num_timesteps = data.shape[1]  # Number of time steps
num_features = data.shape[2]  # Number of features

Hyperparameter Tuning process


In [36]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [10]:
# Define the function to create the model
def create_model(units=64, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units=units, input_shape=(data.shape[1], data.shape[2])))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [11]:
# Create the KerasClassifier wrapper
model = KerasClassifier(build_fn=create_model)


C:\Users\Harsh Bisla\AppData\Local\Temp\ipykernel_5800\1610858891.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


In [12]:
# Define the hyperparameters to search
param_grid = {
    'units': [32, 64, 128],  # Number of LSTM units
}

# Perform the grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(X_test, y_test)

# Print the best parameters and accuracy
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)

3/3 [==============================] - 7s 676ms/step - loss: 0.6517 - accuracy: 0.7391
Best Parameters:  {'units': 64}
Best Accuracy:  0.8272401293118795


In [56]:
# Define the RNN model
model = Sequential()
model.add(LSTM(units=32, input_shape=(data.shape[1], data.shape[2])))

model.add(Dense(units=1, activation='sigmoid'))

In [49]:
# from keras.callbacks import ModelCheckpoint
# import os

In [57]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# if not os.path.exists('models'):
#     os.makedirs('models')
# # Define a checkpoint callback to save the best model based on validation accuracy
# checkpoint = ModelCheckpoint('models/mymodel.h5', monitor='val_accuracy', save_best_only=True, mode='max')

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/50
10/10 [==============================] - 9s 622ms/step - loss: 0.5643 - accuracy: 0.8089 - val_loss: 0.4694 - val_accuracy: 0.8243
Epoch 2/50
10/10 [==============================] - 5s 548ms/step - loss: 0.4848 - accuracy: 0.8123 - val_loss: 0.4714 - val_accuracy: 0.8243
Epoch 3/50
10/10 [==============================] - 5s 461ms/step - loss: 0.4984 - accuracy: 0.8123 - val_loss: 0.4818 - val_accuracy: 0.8243
Epoch 4/50
10/10 [==============================] - 5s 511ms/step - loss: 0.4833 - accuracy: 0.8123 - val_loss: 0.4658 - val_accuracy: 0.8243
Epoch 5/50
10/10 [==============================] - 6s 572ms/step - loss: 0.4828 - accuracy: 0.8123 - val_loss: 0.4651 - val_accuracy: 0.8243
Epoch 6/50
10/10 [==============================] - 5s 547ms/step - loss: 0.4749 - accuracy: 0.8123 - val_loss: 0.4669 - val_accuracy: 0.8243
Epoch 7/50
10/10 [==============================] - 5s 472ms/step - loss: 0.4782 - accuracy: 0.8123 - val_loss: 0.4666 - val_accuracy: 0.8243
Epoch 

In [58]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:f}')
print(f'Test Accuracy: {accuracy:f}')

# Make predictions
predictions = model.predict(X_test)

3/3 [==============================] - 1s 167ms/step - loss: 0.5840 - accuracy: 0.7609
Test Loss: 0.584013
Test Accuracy: 0.760870
3/3 [==============================] - 1s 149ms/step


In [108]:
# # Load the best model from the saved checkpoint
# best_model = keras.models.load_model('models/mymodel.h5')

# # Assuming you have your test data (X_test and y_test) ready
# # If you don't have the test data, prepare it using your dataset

# # Evaluate the best model on the test data
# loss, accuracy = best_model.evaluate(X_test, y_test)

# # Print the accuracy
# print(f'Test accuracy of the best model: {accuracy:.4f}')

3/3 [==============================] - 1s 66ms/step - loss: 0.8620 - accuracy: 0.6739
Test accuracy of the best model: 0.6739
